In [ ]:
import pandas as pd
import re
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/EuroEval/leaderboards/refs/heads/main/leaderboards/european_all.csv"
)
df.head(2)

In [ ]:
columns_to_drop = [col for col in df.columns if col.endswith("_version")]
columns_to_drop += [col.replace("_version", "") for col in columns_to_drop]
columns_to_drop += [
    "generative_type",
    "parameters",
    "vocabulary_size",
    "context",
    "commercial",
    "merge",
    "rank",
]

clean_df = (
    df.map(lambda x: x.split("@@")[0] if isinstance(x, str) else x)
    .map(lambda x: re.sub(r"<.*?>(.*?)</.*>", r"\1", x) if isinstance(x, str) else x)
    .map(lambda x: None if isinstance(x, str) and "few-shot" in x else x)
    .map(lambda x: re.sub(r" \(.*\)$", "", x) if isinstance(x, str) else x)
    .map(lambda x: None if x == "-" else x)
    .map(lambda x: re.sub(r"^(gemini|xai)/", "", x) if isinstance(x, str) else x)
    .drop(columns=columns_to_drop)
    .dropna()
    .set_index("model")
    .map(float)
)
clean_df

In [ ]:
def plot_models(models: list[str], title: str) -> None:
    """Create a spider plot of a list of models."""
    fig = go.Figure()

    for model in models:
        model_scores = clean_df.loc[model, :].tolist()
        trace = go.Scatterpolar(
            r=model_scores,
            theta=[x.capitalize() for x in clean_df.columns],
            name=model,
            fill="toself",
        )
        fig.add_trace(trace)

    fig.update_layout(
        polar=dict(radialaxis=dict(range=[4, 1])),
        showlegend=True,
        title=title.strip() + " (smaller is better)",
        width=800,
        height=500,
    )
    fig.show(config=dict(toImageButtonOptions=dict(scale=6)))

In [ ]:
plot_models(
    models=[
        "gemini-2.5-pro-preview-03-25",
        "gpt-4.1-2025-04-14",
        "claude-3-7-sonnet-20241022",
        # "gemini-2.0-flash-001",
        #'grok-2-1212',
    ],
    title="European Performance of State-of-the-art API LLMs",
)

In [ ]:
plot_models(
    models=[
        "gemini-2.0-flash-lite-001",
        "gpt-4.1-mini-2025-04-14",
        "claude-3-5-haiku-20241022",
    ],
    title="European Performance of Fast and Cheap API LLMs",
)